# Week3授業前課題２　住宅情報の分析

In [313]:
import pandas as pd
import numpy as np
import missingno as msno

### 【問題1】データセットの取得

In [314]:
Ames = pd.read_csv("train.csv")
#print(Ames)

### 【問題2】データセット自体の調査

各住宅の最終価格を予測することを目的としたコンペ用のデータセット。アイオワ州エイムスの住宅に関する７９の説明変数を持つ。テストセットの各Idについて、販売価格変数の値を予測するためのデータセットです。


### 【問題3】データの確認

・各特徴量がどういったものかを見る。（数値データか文字データなど）

In [315]:
# 説明変数（特徴量：data）をXに格納
X = pd.DataFrame(Ames.drop("SalePrice", axis=1))
# print(X.dtypes) # データのタイプを確認
X.head(4)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml


In [316]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

・今回の目的変数はどの列か確認する。

データフィールドより：このデータセットの目的変数はSalePriceと言える<br>
SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.

In [317]:
# 目的変数をyに格納
y = pd.DataFrame(Ames["SalePrice"])
y.head(4)

,SalePrice
0,208500
1,181500
2,223500
3,140000


・特徴量の平均値と標準偏差、四分位数を一度に表示させる。

In [318]:
X.describe() 

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


### 【問題4】欠損値への対応

・各特徴量の欠損値の有無を確認する。

In [319]:
nan_X = X.isnull().sum() # 列ごとの欠損値の合計
nan_X

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
HalfBath            0
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
GarageCond

・欠損値が何パーセント含まれるかを確認する

In [320]:
# 特徴量ごとの欠損値の割合

nan_index= nan_X.index
nan_X = (X.isnull().sum()/len(X)).round(3)
nan_X = pd.DataFrame(nan_X, index=nan_index)
nan_X = nan_X.assign(total = len(X))
nan_X.columns = ["nan(%)", "total"]
nan_X

,nan(%),total
Id,0.000,1460
MSSubClass,0.000,1460
MSZoning,0.000,1460
LotFrontage,0.177,1460
LotArea,0.000,1460
Street,0.000,1460
Alley,0.938,1460
LotShape,0.000,1460
LandContour,0.000,1460
Utilities,0.000,1460


In [321]:
total = np.array([len(X)]*X.shape[1])
print(total)
nan_X = X.isnull().sum()
print(nan_X)
nan_X  = np.concatnate((total,  nan_X), axis=1)
print(nan_X)

[1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460
 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460
 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460
 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460
 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460
 1460 1460 1460 1460 1460 1460 1460 1460 1460 1460]
Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8

AttributeError: module 'numpy' has no attribute 'concatnate'

・5つ以上欠損値がある特徴量（列）は削除する。

In [322]:
#5つ以上欠損値がある列
many_nan_columns = X.isnull().sum()[X.isnull().sum()>=5]

#列名をリストにする
many_nan_columns = many_nan_columns.index

# リストで複数列を指定して削除
#X.drop(many_nan_columns, axis=1)

#print(processed_list)
print(many_nan_columns)

#for i in many_nan_columns:
    #X.drop(many_nan_columns[i], axis=1)
    #i += 1
#X

#5つ以上欠損のある列名のリストを作って列を削除したかったのですが、なぜか消えませんでした
X.drop(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)
X

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype='object')


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


・欠損値があるサンプル（行）は削除する

In [323]:
X = X.dropna(how='any')
X

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


### 【問題5】用語の調査

・歪度（わいど）は、分布の左右対称性の違いを表す。 データの個数 : データ全体の平均値 : データ全体の標準偏差 : 歪度 の値によって分布の左右対称性がわかる。 尖度（せんど）は、分布の形が先がとがっているか偏平かを表す。

・尖度
尖度（せんど、英: kurtosis)は、確率変数の確率密度関数や頻度分布の鋭さを表す数字である。 正規分布と比べて、尖度が大きければ鋭いピークと長く太い裾を持った分布を持ち、尖度が小さければより丸みがかったピークと短く細い尾を持った分布であるという事が判断できる。

確率変数 X が従う分布の尖度，歪度は以下のように定義される：　<br>
歪度：(E[(X−μ)3]) / σ3<br>
尖度：(E[(X−μ)4]) / σ4) −3

### 【問題6】分布の確認

#昨晩、ジュピターが動かなくなってしまい、環境構築からやり直していたため、最後まで回答が完成しませんでした。いったん現状で提出します。申し訳ありません。

### 【問題7】相関係数の確認